In [11]:
magma_dir = '/home/ubuntu/magma/'
transformers_dir = '/home/ubuntu/transformers/'

## **Fine-tuning**

In [13]:
finetune_script = '"'+transformers_dir+'examples/seq2seq/finetune_trainer.py"'
eval_script = '"'+transformers_dir+'examples/seq2seq/run_eval.py"'

### **Config**

In [14]:
import sys
sys.path.insert(0, magma_dir)
import config

import torch
torch.manual_seed = config.SEED

import wandb
wandb.login()

wandb: Currently logged in as: marcoabrate (use `wandb login --relogin` to force relogin)


True

### Karger Books Para

In [15]:
model_name_or_path = 'sshleifer/bart-tiny-random'

In [16]:
data_dir = '"'+magma_dir+'datasets/karger_books_para/bart"'

output_dir = '"'+magma_dir+'fine-tuning/'+\
    model_name_or_path.replace('/', '?')+'_karger_books_para_train"'

log_dir = output_dir + '/logs'

In [17]:
from transformers import AutoConfig
model_config = AutoConfig.from_pretrained(model_name_or_path)
model_config.min_length = config.ONE_BULLET_MIN_LEN
model_config.max_length = config.ONE_BULLET_MAX_LEN
model_config.length_penalty = config.LENGTH_PENALTY
model_config.no_repeat_ngram_size = config.NO_REPEAT_NGRAM_SIZE

model_config.task_specific_params['summarization']['min_length'] = config.ONE_BULLET_MIN_LEN
model_config.task_specific_params['summarization']['max_length'] = config.ONE_BULLET_MAX_LEN
model_config.task_specific_params['summarization']['length_penalty'] = config.LENGTH_PENALTY
model_config.task_specific_params['summarization']['no_repeat_ngram_size'] = config.NO_REPEAT_NGRAM_SIZE
model_config_dir = '"'+magma_dir+'fine-tuning/'+\
    model_name_or_path.replace('/', '?')+'_config"'
model_config.save_pretrained(model_config_dir[1:-1])

Downloading:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

##### Fine tune

In [19]:
!python3 $finetune_script \
--model_name_or_path $model_name_or_path \
--config_name $model_config_dir \
--tokenizer_name $model_name_or_path \
--data_dir $data_dir \
--fp16 \
--learning_rate 3e-5 --label_smoothing 0.1 \
--sortish_sampler --freeze_embeds --adafactor \
--task summarization \
--max_source_length 1024 \
--max_target_length $config.ONE_BULLET_MAX_LEN \
--val_max_target_length $config.ONE_BULLET_MAX_LEN \
--test_max_target_length $config.ONE_BULLET_MAX_LEN \
--do_train \
--num_train_epochs 10 \
--logging_steps 10 --save_steps 1000 --save_total_limit 3 \
--per_device_train_batch_size 4 --per_device_eval_batch_size 4 \
--do_eval --evaluation_strategy steps --eval_steps 1000 --eval_beams 2 \
--metric_for_best_model rougeL --greater_is_better True \
--predict_with_generate \
--output_dir $output_dir \
--overwrite_output_dir \
--seed $config.SEED \
--run_name $output_dir

01/14/2021 14:47:01 - WARNING - __main__ -   Process rank: -1, device: cpu, n_gpu: 0, distributed training: False, 16-bits training: False
01/14/2021 14:47:01 - INFO - __main__ -   Training/evaluation parameters Seq2SeqTrainingArguments(output_dir='/home/marco/epfl/magma/fine-tuning/sshleifer?bart-tiny-random_karger_books_para_train', overwrite_output_dir=True, do_train=True, do_eval=True, do_predict=False, model_parallel=False, evaluation_strategy=<EvaluationStrategy.STEPS: 'steps'>, prediction_loss_only=False, per_device_train_batch_size=4, per_device_eval_batch_size=4, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=1, eval_accumulation_steps=None, learning_rate=3e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=10.0, max_steps=-1, warmup_steps=0, logging_dir='runs/Jan14_14-47-01_calcolatore', logging_first_step=False, logging_steps=10, save_steps=1000, save_total_limit=3, no_cud

[INFO|tokenization_utils_base.py:1802] 2021-01-14 14:47:05,621 >> loading file https://huggingface.co/sshleifer/bart-tiny-random/resolve/main/vocab.json from cache at /home/marco/.cache/huggingface/transformers/6c6a5c8768403bcf7d2da061e943e18fa6d570179aac8951d8df0a45e5e8071e.d67d6b367eb24ab43b08ad55e014cf254076934f71d832bbab9ad35644a375ab
[INFO|tokenization_utils_base.py:1802] 2021-01-14 14:47:05,621 >> loading file https://huggingface.co/sshleifer/bart-tiny-random/resolve/main/merges.txt from cache at /home/marco/.cache/huggingface/transformers/19b7b337f8ee4dea92f336e87e5a4e785eef3ba09323cda02617d338ec06eee0.5d12962c5ee615a4c803841266e9c3be9a691a924f72d395d3a6c6c81157788b
[INFO|tokenization_utils_base.py:1802] 2021-01-14 14:47:05,622 >> loading file https://huggingface.co/sshleifer/bart-tiny-random/resolve/main/tokenizer.json from cache at None
[INFO|tokenization_utils_base.py:1802] 2021-01-14 14:47:05,622 >> loading file https://huggingface.co/sshleifer/bart-tiny-random/resolve/main/

100%|██████████| 250/250 [00:40<00:00,  4.58it/s][INFO|trainer.py:862] 2021-01-14 14:47:49,655 >> 

Training completed. Do not forget to share your model on huggingface.co/models =)


100%|██████████| 250/250 [00:40<00:00,  6.25it/s]
[INFO|trainer.py:1226] 2021-01-14 14:47:49,656 >> Saving model checkpoint to /home/marco/epfl/magma/fine-tuning/sshleifer?bart-tiny-random_karger_books_para_train

[INFO|configuration_utils.py:289] 2021-01-14 14:47:49,662 >> Configuration saved in /home/marco/epfl/magma/fine-tuning/sshleifer?bart-tiny-random_karger_books_para_train/config.json
[INFO|modeling_utils.py:814] 2021-01-14 14:47:49,673 >> Model weights saved in /home/marco/epfl/magma/fine-tuning/sshleifer?bart-tiny-random_karger_books_para_train/pytorch_model.bin
01/14/2021 14:47:49 - INFO - __main__ -   ***** train metrics *****
01/14/2021 14:47:49 - INFO - __main__ -     train_samples_per_second = 2.397
01/14/2021 14:47:49 - INFO - __main__ -     train_runtime = 41.7114
01/14/2021 14:47:49 - IN

In [22]:
!aws s3 mv $output_dir "s3://marcoabrate/output" --recursive
# should do the same with model_config_dir

move: sshleifer?bart-tiny-random_karger_books_para_train/all_results.json to s3://marcoabrate/output/all_results.json
move: sshleifer?bart-tiny-random_karger_books_para_train/special_tokens_map.json to s3://marcoabrate/output/special_tokens_map.json
move: sshleifer?bart-tiny-random_karger_books_para_train/tokenizer_config.json to s3://marcoabrate/output/tokenizer_config.json
move: sshleifer?bart-tiny-random_karger_books_para_train/trainer_state.json to s3://marcoabrate/output/trainer_state.json
move: sshleifer?bart-tiny-random_karger_books_para_train/train_results.json to s3://marcoabrate/output/train_results.json
move: sshleifer?bart-tiny-random_karger_books_para_train/val_results.json to s3://marcoabrate/output/val_results.json
move: sshleifer?bart-tiny-random_karger_books_para_train/config.json to s3://marcoabrate/output/config.json
move: sshleifer?bart-tiny-random_karger_books_para_train/training_args.bin to s3://marcoabrate/output/training_args.bin
move: sshleifer?bart-tiny-random

In [ ]:
!rm -r $output_dir

##### Evaluate

In [ ]:
source_test_dir = data_dir[:-1] + '/test.source"'
reference_test_dir = data_dir[:-1] + '/test.target"'

save_dir = output_dir[:-1] + '/'+model_name_or_path.replace('/', '?')+'_test_karger_books_para.txt"'
score_dir = output_dir[:-1] + '/'+model_name_or_path.replace('/', '?')+'_test_karger_books_para.json"'

In [ ]:
!python3 $eval_script \
$output_dir \
$source_test_dir \
$save_dir \
--reference_path $reference_test_dir \
--score_path $score_dir \
--task summarization \
--bs 2 \
--length_penalty $config.LENAGTH_PENALTY \
--no_repeat_ngram_size $config.NO_REPEAT_NGRAM_SIZE \
--num_beams $config.NUM_BEAMS \
--dump-args

Unknown option: --
usage: python3 [option] ... [-c cmd | -m mod | file | -] [arg] ...
Try `python -h' for more information.
